<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-07-12 21:02:45


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv'
df_mypf = read_file(path_mypf)

#mypf = mypf[mypf['Nifty100'] == 1]

def get_stock_list(df_mypf):
  stock_n100 = df_mypf['Symbol'].unique()
  exclude = ['CADILAHC','MMTC']
  stock_n100 = df_mypf[~df_mypf['Symbol'].isin(exclude) ]['Symbol'].unique()
  stock_ids = stock_n100
  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()
  return stock_ids

stock_ids = get_stock_list(df_mypf)

In [5]:
# star stocks & my stocks
star_stocks = list(df_mypf[df_mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(df_mypf[df_mypf['InPortfolio'] == 1]['Symbol'].values)

In [6]:
def print_distribution():
  # conviction distribution
  print('Conviction:')
  print(df_mypf['Conviction'].value_counts())

  # star stocks
  print('Star Stocks:')
  print(df_mypf['StarStock'].value_counts())

  # v40/v40n stocks
  print('V40/V40N Stocks:')
  print(df_mypf['Criteria'].value_counts())

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())

  # stocks in portfolio
  print('Stocks in Portfolio:')
  print(df_mypf['InPortfolio'].value_counts())

  # star stocks in portfolio
  print('Star Stocks in Portfolio:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['StarStock'].value_counts())

  # criteria of portfolio stocks
  print('Criteria in Portfolio:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Criteria'].value_counts())

  # strategies in portfolio stocks
  print('Strategy in Portfolio:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Strategy'].value_counts())

print_distribution()

Conviction:
M    77
L    72
H    37
Name: Conviction, dtype: int64
Star Stocks:
0.0    95
1.0    89
Name: StarStock, dtype: int64
V40/V40N Stocks:
V40N    39
V40     38
2T3Y    11
SS       9
V200     6
TBD      3
ATH      1
Name: Criteria, dtype: int64
Latest Qtr:
0    186
Name: LatestQtr, dtype: int64
Star + V40/V40N Stocks:
1.0    41
0.0    36
Name: StarStock, dtype: int64
Stocks in Portfolio:
0    140
1     46
Name: InPortfolio, dtype: int64
Star Stocks in Portfolio:
1.0    32
0.0    14
Name: StarStock, dtype: int64
Criteria in Portfolio:
V40N    11
2T3Y    10
V40     10
SS       8
V200     3
TBD      3
ATH      1
Name: Criteria, dtype: int64
Strategy in Portfolio:
ATH     31
2T3Y    14
Name: Strategy, dtype: int64


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000

  return current_pe


In [9]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)

    stock_df.ta.rsi(append=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)
    stock_df['RSI'] = round(stock_df['RSI'],0)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    tmp['Curr_PE'] = current_pe
    #tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0]
    #tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]
    tmp['Conviction'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
    tmp['LatestQtr'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
    tmp['StarStock'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
    tmp['Criteria'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

In [10]:

# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,,
2023-06-13 00:00:00+05:30,18716.0,^NSEI,4.67,0.26,0.90,66.0,19497.0,7610.0,6.57,4.0
2023-06-20 00:00:00+05:30,18817.0,^NSEI,5.05,0.29,0.75,65.0,19497.0,7610.0,5.72,4.0
2023-06-27 00:00:00+05:30,18817.0,^NSEI,4.88,0.33,0.56,61.0,19497.0,7610.0,5.72,4.0
2023-07-05 00:00:00+05:30,19398.0,^NSEI,7.91,0.36,0.86,76.0,19497.0,7610.0,0.83,1.0
2023-07-12 00:00:00+05:30,19384.0,^NSEI,7.59,0.42,0.78,68.0,19497.0,7610.0,0.95,1.0


In [11]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

df_mypf['Stock'] = df_mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [12]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [13]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'RSI', 'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 46


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,RSI,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,,
PAGEIND.NS,36005.0,-14.91,-1.41,-1.02,46.84,49.0,38548.77,97.7,70.30,27.0,H,28.0,0,1.0,V40,ATH
CLEAN.NS,1309.0,-11.22,-1.76,-0.35,95.03,102.0,1396.49,NaN,47.10,30.0,L,NaN,0,1.0,2T3Y,2T3Y
BANDHANBNK.NS,222.0,-7.24,-1.26,0.00,88.05,226.0,253.98,NaN,1000.00,32.0,M,NaN,0,1.0,2T3Y,2T3Y
VALIANTORG.NS,505.0,-9.74,-1.79,0.37,91.50,267.0,542.52,24.0,16.20,33.0,L,33.0,0,0.0,2T3Y,2T3Y
FINEORG.NS,4676.0,-11.61,-1.85,0.92,39.00,54.0,4697.43,NaN,23.20,38.0,M,NaN,0,1.0,V40N,ATH
EICHERMOT.NS,3292.0,-2.18,-0.18,0.00,21.48,17.0,3260.20,44.7,30.90,38.0,H,31.0,0,1.0,V40N,ATH
RAJESHEXPO.NS,527.0,-21.02,-0.30,-0.53,82.91,80.0,624.78,NaN,10.90,39.0,M,NaN,0,1.0,V40N,ATH
LTIM.NS,4816.0,4.38,0.67,1.71,41.71,53.0,4996.37,26.6,32.30,39.0,M,-21.0,0,1.0,V200,ATH
IRCTC.NS,621.0,-5.09,-0.61,0.31,52.92,87.0,633.05,77.4,50.30,40.0,M,35.0,0,1.0,SS,ATH


In [14]:
# top v40/v40n stocks from portfolio for SIP
tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')][tmp_df['Dev%_200'] < 0].sort_values(by = 'RSI', ascending=True)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,RSI,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,,
PAGEIND.NS,36005.0,-14.91,-1.41,-1.02,46.84,49.0,38548.77,97.7,70.3,27.0,H,28.0,0,1.0,V40,ATH
EICHERMOT.NS,3292.0,-2.18,-0.18,0.00,21.48,17.0,3260.20,44.7,30.9,38.0,H,31.0,0,1.0,V40N,ATH
FINEORG.NS,4676.0,-11.61,-1.85,0.92,39.00,54.0,4697.43,NaN,23.2,38.0,M,NaN,0,1.0,V40N,ATH
RAJESHEXPO.NS,527.0,-21.02,-0.30,-0.53,82.91,80.0,624.78,NaN,10.9,39.0,M,NaN,0,1.0,V40N,ATH
LUXIND.NS,1483.0,-2.61,-1.25,1.01,82.85,207.0,1493.72,21.4,32.4,47.0,M,-51.0,0,0.0,V40N,2T3Y
TEAMLEASE.NS,2407.0,-3.21,-1.69,1.14,75.61,119.0,2279.93,NaN,36.3,55.0,M,NaN,0,1.0,V40N,2T3Y
INFY.NS,1333.0,-5.10,-0.28,1.02,37.65,39.0,1263.00,29.0,23.1,61.0,H,20.0,0,1.0,V40,ATH


In [15]:
# top 10 from portfolio for SIP
tmp_df[tmp_df['Dev%_200'] < 0].sort_values(by = 'RSI', ascending=True).head(10)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,RSI,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,,
PAGEIND.NS,36005.0,-14.91,-1.41,-1.02,46.84,49.0,38548.77,97.7,70.3,27.0,H,28.0,0,1.0,V40,ATH
CLEAN.NS,1309.0,-11.22,-1.76,-0.35,95.03,102.0,1396.49,NaN,47.1,30.0,L,NaN,0,1.0,2T3Y,2T3Y
BANDHANBNK.NS,222.0,-7.24,-1.26,0.00,88.05,226.0,253.98,NaN,1000.0,32.0,M,NaN,0,1.0,2T3Y,2T3Y
VALIANTORG.NS,505.0,-9.74,-1.79,0.37,91.50,267.0,542.52,24.0,16.2,33.0,L,33.0,0,0.0,2T3Y,2T3Y
EICHERMOT.NS,3292.0,-2.18,-0.18,0.00,21.48,17.0,3260.20,44.7,30.9,38.0,H,31.0,0,1.0,V40N,ATH
FINEORG.NS,4676.0,-11.61,-1.85,0.92,39.00,54.0,4697.43,NaN,23.2,38.0,M,NaN,0,1.0,V40N,ATH
RAJESHEXPO.NS,527.0,-21.02,-0.30,-0.53,82.91,80.0,624.78,NaN,10.9,39.0,M,NaN,0,1.0,V40N,ATH
IRCTC.NS,621.0,-5.09,-0.61,0.31,52.92,87.0,633.05,77.4,50.3,40.0,M,35.0,0,1.0,SS,ATH
VAIBHAVGBL.NS,301.0,-4.92,-1.26,-0.32,77.42,221.0,317.48,35.4,47.4,41.0,L,-34.0,0,0.0,2T3Y,2T3Y


In [16]:
# top 5 from portfolio to book profit
tmp_df.sort_values(by = 'Upside%ATH', ascending=True).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,RSI,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,,
ASIANPAINT.NS,3407.0,12.43,-0.03,1.54,5.52,4.0,3091.34,87.9,79.1,68.0,H,10.0,0,1.0,V40,ATH
PIDILITIND.NS,2622.0,3.65,-0.47,0.77,13.97,11.0,2504.06,94.6,105.0,52.0,H,-11.0,0,1.0,V40,ATH
EICHERMOT.NS,3292.0,-2.18,-0.18,0.00,21.48,17.0,3260.20,44.7,30.9,38.0,H,31.0,0,1.0,V40N,ATH
TCS.NS,3260.0,1.33,0.37,0.43,25.75,18.0,3231.00,28.1,27.3,51.0,H,3.0,0,1.0,V40,ATH
ICICIGI.NS,1362.0,17.64,0.43,2.25,27.31,18.0,1137.80,46.2,1000.0,73.0,H,-2065.0,0,1.0,V40,ATH
BAJAJFINSV.NS,1593.0,7.07,-0.60,1.83,20.86,20.0,1454.93,39.2,39.5,67.0,H,-1.0,0,1.0,V40,ATH
BAYERCROP.NS,4675.0,5.45,-0.56,1.24,41.01,28.0,4185.24,NaN,427.0,80.0,M,NaN,0,1.0,V40N,ATH


In [17]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)
tmp_base_df = df_prec_dev[~df_prec_dev.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]

In [18]:
# star + v40/v40n stocks (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 26)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,
DIXON.NS,4336.0,18.45,-0.22,3.64,59.0,5750.0,374.0,26.30,33.0,M,0,1.0,V40N,NaN
BERGEPAINT.NS,669.0,10.61,0.17,1.08,53.0,858.0,260.0,31.61,28.0,M,0,1.0,V40,NaN


In [19]:
# star + v40/v40n stocks (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,
3MINDIA.NS,28455.0,21.16,1.12,2.02,66.0,29373.0,15332.0,6.54,3.0,M,0,1.0,V40N,NaN
HINDUNILVR.NS,2674.0,4.28,0.27,0.95,51.0,2757.0,1394.0,6.09,3.0,H,0,1.0,V40,NaN
ICICIBANK.NS,944.0,5.28,0.22,0.43,54.0,959.0,252.0,2.12,2.0,H,0,1.0,V40,NaN
RELIANCE.NS,2768.0,12.28,0.08,1.15,78.0,2811.0,867.0,2.21,2.0,H,0,1.0,V40,ATH
TITAN.NS,3096.0,18.37,0.80,1.64,67.0,3144.0,730.0,1.99,2.0,H,0,1.0,V40,NaN
ABBOTINDIA.NS,23564.0,12.53,1.25,0.54,64.0,23826.0,6634.0,1.52,1.0,M,0,1.0,V40,NaN
BAJAJ-AUTO.NS,4878.0,29.99,1.49,1.77,71.0,4918.0,1695.0,1.24,1.0,L,0,1.0,V40,NaN
ITC.NS,472.0,26.28,1.87,1.36,69.0,475.0,124.0,0.85,1.0,H,0,1.0,V40,NaN
AKZOINDIA.NS,2723.0,19.67,1.05,1.14,81.0,2723.0,1360.0,0.00,0.0,M,0,1.0,V40,NaN


In [20]:
# star stocks (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 7


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,
UPL.NS,644.0,-10.33,-0.42,-1.02,31.0,832.0,245.0,32.03,29.0,L,0,1.0,NaN,NaN
SBICARD.NS,838.0,3.44,-0.74,0.80,34.0,1133.0,496.0,46.31,35.0,M,0,1.0,NaN,NaN
ADANITRANS.NS,753.0,-59.36,-8.69,-2.89,35.0,4105.0,148.0,84.71,445.0,L,0,1.0,NaN,NaN
ADANIGREEN.NS,955.0,-31.96,-4.92,0.32,49.0,2970.0,30.0,68.54,211.0,L,0,1.0,NaN,NaN
GODREJIND.NS,497.0,11.99,0.23,1.25,52.0,644.0,235.0,35.94,30.0,L,0,1.0,NaN,NaN
ADANIENT.NS,2388.0,-13.70,-2.17,2.41,53.0,4161.0,83.0,43.48,74.0,M,0,1.0,NaN,NaN
TATAPOWER.NS,226.0,7.76,-0.48,1.40,62.0,285.0,26.0,22.78,26.0,M,0,1.0,NaN,NaN


In [21]:
# star stocks (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,
AKZOINDIA.NS,2723.0,19.67,1.05,1.14,81.0,2723.0,1360.0,0.00,0.0,M,0,1.0,V40,NaN
COLPAL.NS,1798.0,16.15,0.45,1.41,75.0,1798.0,931.0,0.00,0.0,H,0,1.0,V40,NaN
EQUITASBNK.NS,94.0,44.38,3.08,2.38,66.0,94.0,33.0,0.00,0.0,M,0,1.0,V40N,NaN
POLYCAB.NS,3800.0,28.96,1.70,1.25,72.0,3800.0,523.0,0.00,0.0,M,0,1.0,V40N,NaN
ABBOTINDIA.NS,23564.0,12.53,1.25,0.54,64.0,23826.0,6634.0,1.52,1.0,M,0,1.0,V40,NaN
BAJAJ-AUTO.NS,4878.0,29.99,1.49,1.77,71.0,4918.0,1695.0,1.24,1.0,L,0,1.0,V40,NaN
ITC.NS,472.0,26.28,1.87,1.36,69.0,475.0,124.0,0.85,1.0,H,0,1.0,V40,NaN
ICICIBANK.NS,944.0,5.28,0.22,0.43,54.0,959.0,252.0,2.12,2.0,H,0,1.0,V40,NaN
RELIANCE.NS,2768.0,12.28,0.08,1.15,78.0,2811.0,867.0,2.21,2.0,H,0,1.0,V40,ATH


In [22]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,
NESTLEIND.NS,23060.0,14.65,1.00,0.83,60.0,23099.0,8621.0,0.27,0.0,H,0,0.0,V40,NaN
PGHH.NS,14902.0,7.25,0.09,0.57,68.0,15778.0,8145.0,11.48,6.0,L,0,0.0,V40,NaN


In [23]:
# stocks prospects

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 0


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,


In [24]:
# prospects with heavy upside
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 10


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,
SYMPHONY.NS,882.0,-6.32,-0.11,-1.23,40.0,1470.0,707.0,77.06,67.0,M,0,0.0,V40N,NaN
TATAELXSI.NS,7431.0,8.12,-0.90,1.57,46.0,10602.0,525.0,31.47,43.0,M,0,0.0,V40N,NaN
PFIZER.NS,3809.0,-5.64,-0.57,0.08,50.0,6009.0,2301.0,59.33,58.0,L,0,0.0,V40,NaN
KANSAINER.NS,303.0,6.58,-0.35,2.43,51.0,436.0,192.0,54.51,44.0,M,0,0.0,V40N,NaN
JCHAC.NS,1057.0,-8.10,-2.26,-0.75,52.0,3219.0,1017.0,98.18,205.0,M,0,0.0,V40N,NaN
WHIRLPOOL.NS,1455.0,0.98,-1.04,0.99,52.0,2667.0,1254.0,85.77,83.0,M,0,0.0,V40,NaN
SFL.NS,1169.0,-3.33,-1.49,1.28,56.0,1972.0,566.0,57.11,69.0,M,0,0.0,V40N,NaN
TTKPRESTIG.NS,753.0,-5.69,-1.63,0.42,58.0,1165.0,391.0,53.23,55.0,M,0,0.0,V40N,NaN
MOTILALOFS.NS,742.0,12.03,-0.30,1.82,64.0,1091.0,439.0,53.53,47.0,L,0,0.0,V40N,NaN


In [25]:
# prospects near 52W low
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 3


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,
SYMPHONY.NS,882.0,-6.32,-0.11,-1.23,40.0,1470.0,707.0,77.06,67.0,M,0,0.0,V40N,NaN
JCHAC.NS,1057.0,-8.10,-2.26,-0.75,52.0,3219.0,1017.0,98.18,205.0,M,0,0.0,V40N,NaN
WHIRLPOOL.NS,1455.0,0.98,-1.04,0.99,52.0,2667.0,1254.0,85.77,83.0,M,0,0.0,V40,NaN


In [26]:
# prospects near 52W high
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 25


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,
AKZOINDIA.NS,2723.0,19.67,1.05,1.14,81.0,2723.0,1360.0,0.00,0.0,M,0,1.0,V40,NaN
RADICO.NS,1393.0,25.08,1.08,1.93,75.0,1393.0,241.0,0.00,0.0,M,0,0.0,V40N,NaN
BAJAJHLDNG.NS,7401.0,15.63,0.58,1.03,70.0,7401.0,1451.0,0.00,0.0,L,0,0.0,V40,NaN
POLYCAB.NS,3800.0,28.96,1.70,1.25,72.0,3800.0,523.0,0.00,0.0,M,0,1.0,V40N,NaN
COLPAL.NS,1798.0,16.15,0.45,1.41,75.0,1798.0,931.0,0.00,0.0,H,0,1.0,V40,NaN
EQUITASBNK.NS,94.0,44.38,3.08,2.38,66.0,94.0,33.0,0.00,0.0,M,0,1.0,V40N,NaN
NESTLEIND.NS,23060.0,14.65,1.00,0.83,60.0,23099.0,8621.0,0.27,0.0,H,0,0.0,V40,NaN
ITC.NS,472.0,26.28,1.87,1.36,69.0,475.0,124.0,0.85,1.0,H,0,1.0,V40,NaN
BAJAJ-AUTO.NS,4878.0,29.99,1.49,1.77,71.0,4918.0,1695.0,1.24,1.0,L,0,1.0,V40,NaN


In [27]:
# prospects in bull run
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 25


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,
ANGELONE.NS,1748.0,29.64,0.89,3.52,69.0,1889.0,207.0,8.38,8.0,M,0,1.0,V40N,NaN
ISEC.NS,610.0,22.02,1.00,3.23,72.0,829.0,164.0,32.93,36.0,M,0,0.0,V40N,NaN
HDFCLIFE.NS,672.0,21.91,0.73,2.35,67.0,754.0,339.0,19.76,12.0,H,0,0.0,V40,NaN
BAJFINANCE.NS,7432.0,13.62,0.15,2.28,58.0,7860.0,1804.0,7.07,6.0,H,0,1.0,V40,ATH
3MINDIA.NS,28455.0,21.16,1.12,2.02,66.0,29373.0,15332.0,6.54,3.0,M,0,1.0,V40N,NaN
SANOFI.NS,6930.0,22.14,1.08,1.94,58.0,8070.0,4096.0,28.69,16.0,M,0,0.0,V40,NaN
RADICO.NS,1393.0,25.08,1.08,1.93,75.0,1393.0,241.0,0.00,0.0,M,0,0.0,V40N,NaN
CAPLIPOINT.NS,822.0,15.52,0.28,1.85,65.0,947.0,190.0,16.51,15.0,M,0,1.0,V40N,NaN
CERA.NS,7141.0,16.89,1.60,1.82,32.0,8036.0,2065.0,14.99,13.0,H,0,1.0,V40N,NaN


In [28]:
# prospects about to turn around
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 15


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,
VIPIND.NS,593.0,-9.49,-0.15,0.16,33.0,743.0,200.0,27.62,25.0,M,0,0.0,V40N,NaN
VINATIORGA.NS,1811.0,-6.52,-1.19,-0.86,41.0,2342.0,453.0,28.11,29.0,M,0,0.0,V40N,NaN
HAVELLS.NS,1291.0,5.19,-0.24,0.62,46.0,1456.0,451.0,16.42,13.0,H,0,0.0,V40,NaN
TATAELXSI.NS,7431.0,8.12,-0.90,1.57,46.0,10602.0,525.0,31.47,43.0,M,0,0.0,V40N,NaN
PFIZER.NS,3809.0,-5.64,-0.57,0.08,50.0,6009.0,2301.0,59.33,58.0,L,0,0.0,V40,NaN
KANSAINER.NS,303.0,6.58,-0.35,2.43,51.0,436.0,192.0,54.51,44.0,M,0,0.0,V40N,NaN
JCHAC.NS,1057.0,-8.10,-2.26,-0.75,52.0,3219.0,1017.0,98.18,205.0,M,0,0.0,V40N,NaN
WHIRLPOOL.NS,1455.0,0.98,-1.04,0.99,52.0,2667.0,1254.0,85.77,83.0,M,0,0.0,V40,NaN
KOTAKBANK.NS,1878.0,2.07,-0.22,-0.16,53.0,2210.0,1050.0,28.62,18.0,L,0,0.0,V40,NaN


In [29]:
# cross-check the features
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'RSI', ascending=True)

qualified stocks: 10


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,
SYMPHONY.NS,882.0,-6.32,-0.11,-1.23,40.0,1470.0,707.0,77.06,67.0,M,0,0.0,V40N,NaN
TATAELXSI.NS,7431.0,8.12,-0.90,1.57,46.0,10602.0,525.0,31.47,43.0,M,0,0.0,V40N,NaN
PFIZER.NS,3809.0,-5.64,-0.57,0.08,50.0,6009.0,2301.0,59.33,58.0,L,0,0.0,V40,NaN
KANSAINER.NS,303.0,6.58,-0.35,2.43,51.0,436.0,192.0,54.51,44.0,M,0,0.0,V40N,NaN
JCHAC.NS,1057.0,-8.10,-2.26,-0.75,52.0,3219.0,1017.0,98.18,205.0,M,0,0.0,V40N,NaN
WHIRLPOOL.NS,1455.0,0.98,-1.04,0.99,52.0,2667.0,1254.0,85.77,83.0,M,0,0.0,V40,NaN
SFL.NS,1169.0,-3.33,-1.49,1.28,56.0,1972.0,566.0,57.11,69.0,M,0,0.0,V40N,NaN
TTKPRESTIG.NS,753.0,-5.69,-1.63,0.42,58.0,1165.0,391.0,53.23,55.0,M,0,0.0,V40N,NaN
MOTILALOFS.NS,742.0,12.03,-0.30,1.82,64.0,1091.0,439.0,53.53,47.0,L,0,0.0,V40N,NaN


In [30]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,
PVRINOX.NS,1421.0,-11.22,-0.81,-0.84,51.0,1917.0,1348.0,87.17,35.0,L,0,0.0,NaN,NaN


In [31]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [32]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BHARTIARTL.NS')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,,
2023-06-12 00:00:00+05:30,836.0,BHARTIARTL.NS,5.87,0.76,1.01,60.0,890.0,258.0,8.54,6.0
2023-06-19 00:00:00+05:30,828.0,BHARTIARTL.NS,4.56,0.63,0.88,54.0,890.0,258.0,9.81,7.0
2023-06-26 00:00:00+05:30,852.0,BHARTIARTL.NS,7.16,0.63,0.99,64.0,890.0,258.0,6.01,4.0
2023-07-04 00:00:00+05:30,865.0,BHARTIARTL.NS,8.41,0.75,1.34,63.0,890.0,258.0,3.96,3.0
2023-07-12 00:00:00+05:30,890.0,BHARTIARTL.NS,11.24,0.62,1.09,72.0,890.0,258.0,0.00,0.0
